# Weather percipitation forecast for the following day

**TODO**: Describe the project

## Data load

In [ ]:
# dependencies

import numpy as np
import pandas as pd
import seaborn as sns

# setup options

pd.set_option('display.max_columns', 30)

In [ ]:
%%bash

# extract data

unzip -o ../data/australia_rain.zip -d ../data/

In [ ]:
# load data from excel file and store to parquet file

column_names = [
    'Date',
    'Location',
    'MinTemp',
    'MaxTemp',
    'Rainfall',
    'Evaporation',
    'Sunshine',
    'WindGustDir',
    'WindGustSpeed',
    'WindDir9am',
    'WindDir3pm',
    'WindSpeed9am',
    'WindSpeed3pm',
    'Humidity9am',
    'Humidity3pm',
    'Pressure9am',
    'Pressure3pm',
    'Cloud9am',
    'Cloud3pm',
    'Temp9am',
    'Temp3pm',
    'RainToday',
    'RISK_MM',
    'RainTomorrow',
]

df = pd.read_excel(
    io='../data/australia_weather.xlsx',
    skiprows=10,
    nrows=156412,
    sheet_name='Sheet1',
    usecols='B:Y',
    index_col=None,
    parse_dates=True,
    header=None,
    names=column_names,
)

df.to_parquet('../data/australia_weather.parquet')

In [ ]:
df = pd.read_parquet('../data/australia_weather.parquet')

df.info()
df.head()


# Exploratory data analysis

# Preprocessing

# Model selection

# Model explanation

# Model training

# Model interpretation

# Model evaluation

# Results summary